In [66]:
# Dependencies
import pandas as pd
from bs4 import BeautifulSoup
import requests
import pymongo
import time
import datetime

In [67]:
# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [74]:
# Define database and collection
db = client.housing_db
collection = db.housing
collection1 = db.redfin

In [2]:
# URL of page to be scraped
url = 'https://www.redfin.com/news/data-center/'

# Retrieve page with the requests module
response = requests.get(url)

# Create BeautifulSoup object; parse with 'html.parser'
soup = BeautifulSoup(response.text, 'html.parser')
soup

<!DOCTYPE html>

<html lang="en-US">
<head>
<meta charset="utf-8"/>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<link href="https://gmpg.org/xfn/11" rel="profile"/>
<!-- This site is optimized with the Yoast SEO Premium plugin v15.4 - https://yoast.com/wordpress/plugins/seo/ -->
<title>Downloadable Housing Market Data - Redfin</title><link href="https://fonts.googleapis.com/css?family=Libre%20Franklin%3A400%2C%7CRoboto%3A100%2C100italic%2C200%2C200italic%2C300%2C300italic%2C400%2C400italic%2C500%2C500italic%2C600%2C600italic%2C700%2C700italic%2C800%2C800italic%2C900%2C900italic%7CRoboto%20Slab%3A100%2C100italic%2C200%2C200italic%2C300%2C300italic%2C400%2C400italic%2C500%2C500italic%2C600%2C600italic%2C700%2C700italic%2C800%2C800italic%2C900%2C900italic&amp;display=swap" rel="stylesheet">
<meta content="View and download the latest housing market data from Redfin, including home prices, sales, inventory, new listings, and days on market." name="description">
<m

In [3]:
# Locate the desired  'a' tags
one_a_tag= soup.findAll('a',target="_blank")[1]
one_a_tag

<a href="https://redfin-public-data.s3-us-west-2.amazonaws.com/redfin_covid19/weekly_housing_market_data_most_recent.tsv" rel="noopener" target="_blank">here</a>

In [4]:
# Extract the actual link
download_url = one_a_tag['href']
download_url

'https://redfin-public-data.s3-us-west-2.amazonaws.com/redfin_covid19/weekly_housing_market_data_most_recent.tsv'

In [33]:
# Load into a pandas dataframe
data = pd.read_csv(download_url, sep ='\t')
data

,region_id,region_type_id,region_name,region_type,period_begin,period_end,duration,total_homes_sold,total_homes_sold_yoy,average_homes_sold,...,pending_sales_to_sales_ratio_yoy,months_of_supply,months_of_supply_yoy,average_pending_sales_listing_updates,average_pending_sales_listing_updates_yoy,percent_total_price_drops_of_inventory,percent_total_price_drops_of_inventory_yoy,percent_homes_sold_above_list,percent_homes_sold_above_list_yoy,last_updated
0,2050,5,"Haywood County, NC",county,2019-09-09,2019-09-15,1 weeks,34.0,1.000000,34.0,...,0.058824,17.882353,-17.176471,32.0,0.230769,0.036778,-0.010179,0.117647,-0.176471,2021-04-21 15:09:35
1,1370,5,"Eaton County, MI",county,2018-11-05,2018-11-11,1 weeks,19.0,0.117647,19.0,...,0.195046,17.631579,-2.191950,NaN,NaN,NaN,NaN,0.473684,0.356037,2021-04-21 15:09:35
2,2455,5,"Greenville County, SC",county,2020-07-20,2020-07-26,1 weeks,207.0,0.196532,207.0,...,-0.035604,13.420290,-4.793583,243.0,0.075221,0.028895,-0.023238,0.198068,0.013097,2021-04-21 15:09:35
3,3175,5,"Dane County, WI",county,2019-02-04,2019-02-10,1 weeks,65.0,-0.015152,65.0,...,-0.352214,25.215385,2.079021,168.0,0.076923,0.022698,0.003412,0.153846,-0.043124,2021-04-21 15:09:35
4,1295,5,"Aroostook County, ME",county,2017-05-01,2017-05-07,1 weeks,8.0,-0.333333,8.0,...,0.291667,57.375000,7.041667,11.0,0.000000,0.026786,-0.013686,0.125000,0.041667,2021-04-21 15:09:35
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1059310,526,5,"Catoosa County, GA",county,2017-03-20,2017-06-11,12 weeks,240.0,0.142857,20.0,...,-0.056426,0.167140,-0.050815,NaN,NaN,NaN,NaN,0.167385,0.005789,2021-04-21 15:09:35
1059311,861,5,"Madison County, IN",county,2018-07-02,2018-09-23,12 weeks,407.0,-0.021635,33.0,...,0.184877,12.024652,-3.360639,42.0,-0.106383,0.871508,0.199132,0.159338,0.008557,2021-04-21 15:09:35
1059312,2068,5,"Montgomery County, NC",county,2020-09-14,2020-12-06,12 weeks,79.0,0.039474,6.0,...,0.100908,18.081884,-15.183393,8.0,0.333333,0.292929,-0.029856,0.061941,0.048052,2021-04-21 15:09:35
1059313,38940,-2,"Port St. Lucie, FL metro area",metro,2017-04-10,2017-07-02,12 weeks,3075.0,0.021255,256.0,...,0.018136,17.420051,-0.739684,315.0,-0.070796,0.440781,-0.837621,0.095506,-0.030455,2021-04-21 15:09:35


In [34]:
list(data.columns)

['region_id',
 'region_type_id',
 'region_name',
 'region_type',
 'period_begin',
 'period_end',
 'duration',
 'total_homes_sold',
 'total_homes_sold_yoy',
 'average_homes_sold',
 'average_homes_sold_yoy',
 'total_homes_sold_with_price_drops',
 'total_homes_sold_with_price_drops_yoy',
 'average_homes_sold_with_price_drops',
 'average_homes_sold_with_price_drops_yoy',
 'percent_homes_sold_with_price_drops',
 'percent_homes_sold_with_price_drops_yoy',
 'median_sale_price',
 'median_sale_price_yoy',
 'median_sale_ppsf',
 'median_sale_ppsf_yoy',
 'median_days_to_close',
 'median_days_to_close_yoy',
 'price_drops',
 'price_drops_yoy',
 'percent_active_listings_with_price_drops',
 'percent_active_listings_with_price_drops_yoy',
 'pending_sales',
 'pending_sales_yoy',
 'median_pending_sqft',
 'median_pending_sqft_yoy',
 'off_market_in_two_weeks',
 'off_market_in_two_weeks_yoy',
 'off_market_in_one_week',
 'off_market_in_one_week_yoy',
 'percent_off_market_in_two_weeks',
 'percent_off_market_i

In [72]:
data.reset_index(inplace = True)
data_dict1 = data.to_dict("records")

In [ ]:
 # Insert dictionary into MongoDB as a document
    collection1.insert_many(data_dict1)

In [56]:
homeprices_df = data[['period_end','region_type','region_name','total_homes_sold','total_homes_sold_yoy','median_sale_price','median_sale_price_yoy','inventory','percent_homes_sold_above_list','median_sale_ppsf']]
homeprices_df.head()

,period_end,region_type,region_name,total_homes_sold,total_homes_sold_yoy,median_sale_price,median_sale_price_yoy,inventory,percent_homes_sold_above_list,median_sale_ppsf
0,2019-09-15,county,"Haywood County, NC",34.0,1.000000,240000.0,-0.008264,571.0,0.117647,143.202606
1,2018-11-11,county,"Eaton County, MI",19.0,0.117647,187500.0,0.102941,316.0,0.473684,121.765601
2,2020-07-26,county,"Greenville County, SC",207.0,0.196532,255000.0,-0.039620,2561.0,0.198068,121.042831
3,2019-02-10,county,"Dane County, WI",65.0,-0.015152,274000.0,-0.086138,1542.0,0.153846,147.583893
4,2017-05-07,county,"Aroostook County, ME",8.0,-0.333333,58700.0,-0.143066,448.0,0.125000,45.265127


In [69]:
homeprices_df = homeprices_df.rename(columns={"period_end":"Month of Period End","region_type":"Region Type","region_name":"Region","total_homes_sold": "Homes Sold","total_homes_sold_yoy":"Homes Sold YOY","median_sale_price":"Median Sale Price",
                                                  "median_sale_price_yoy":"Median Sale Price YOY","inventory":"Inventory",
                                                   "percent_homes_sold_above_list": "Homes Sold Above List price","median_sale_ppsf":"Price Per Square Foot"})
homeprices_df.head()

,index,Month of Period End,Region Type,Region,Homes Sold,Homes Sold YOY,Median Sale Price,Median Sale Price YOY,Inventory,Homes Sold Above List price,Price Per Square Foot
0,0,2019-09-15,county,"Haywood County, NC",34.0,1.000000,240000.0,-0.008264,571.0,0.117647,143.202606
1,1,2018-11-11,county,"Eaton County, MI",19.0,0.117647,187500.0,0.102941,316.0,0.473684,121.765601
2,2,2020-07-26,county,"Greenville County, SC",207.0,0.196532,255000.0,-0.039620,2561.0,0.198068,121.042831
3,3,2019-02-10,county,"Dane County, WI",65.0,-0.015152,274000.0,-0.086138,1542.0,0.153846,147.583893
4,4,2017-05-07,county,"Aroostook County, ME",8.0,-0.333333,58700.0,-0.143066,448.0,0.125000,45.265127


In [58]:
homeprices_df.reset_index(inplace = True)
data_dict = homeprices_df.to_dict("records")

In [70]:
 # Insert dictionary into MongoDB as a document
    collection.insert_many(data_dict)